In [2]:
import numpy as np
import random as r
from engine import Tensor

In [3]:
a = Tensor([[[1.,2,3],[4,5,6]]])
b = a.transpose((2,1,0))
print(a)
print(b)

Tensor:
([[[1. 2. 3.]
  [4. 5. 6.]]])
Grad:
([[[0. 0. 0.]
  [0. 0. 0.]]])
Tensor:
([[[1.]
  [4.]]

 [[2.]
  [5.]]

 [[3.]
  [6.]]])
Grad:
([[[0.]
  [0.]]

 [[0.]
  [0.]]

 [[0.]
  [0.]]])


In [4]:
c = a.transpose()
print(c)

(1, 2, 3)
(0, 2, 1)
Tensor:
([[[1. 4.]
  [2. 5.]
  [3. 6.]]])
Grad:
([[[0. 0.]
  [0. 0.]
  [0. 0.]]])


In [7]:
type(a.shape)

tuple

In [9]:
print(a.shape[:-2] + (a.shape[-1], a.shape[-2]))

(2, 2, 2)


In [11]:
a.shape[:-2]

(2,)

In [13]:
(a.shape[-1], a.shape[-2])

(2, 2)